# **Result Analysis -** SimpleTown Topological Streams

___
___

### **0. Streams**

**Topological Stream 1 :**

- simpletown-maze_0
- simpletown-maze_3
- simpletown-maze_0
- simpletown-maze_2

**Topological Stream 2 :**

- simpletown-maze_0
- simpletown-maze_7
- simpletown-maze_4
- simpletown-maze_7

___

### **1. Initialization**

#### *1.A. Imports*

In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import os
import pandas as pd
import seaborn as sns

from offbench.utils.logger import extract_metrics_from_event_file
from offbench.utils.paths import WORKSPACE_PATH
from hydra import compose, initialize
from omegaconf import DictConfig
from typing import Union

#### *1.B. Parameters*

In [2]:
if True:

    all_experiments_folder_path = os.path.join(WORKSPACE_PATH, "experiments", "2_continual", "godot_goal")

    seeds = [100 * i for i in range(1, 4)]

    max_gradient_steps = 100000

    n_tasks = 4

    stream_names = [
        "simpletown_topological_1", 
        "simpletown_topological_2",
    ]

    stream_name_to_title = {
        "simpletown_topological_1": "SimpleTown - Topological Stream 1",
        "simpletown_topological_2": "SimpleTown - Topological Stream 2",
    }

    agent_ids = [
        "hgcbc_scratch_1_mlp",
        "hgcbc_scratch_n_mlp",
        "hgcbc_finetune_1_mlp",
        "hgcbc_finetune_n_mlp",
        "hgcbc_freeze_mlp",
        "hgcbc_pnn_mlp",
        "hgcbc_replay_mlp",
    ]

    reg_lbs = [0.01, 0.1, 1.0, 10.0, 100.0]
    agent_ids += ["hgcbc_ewc_mlp_lb{}".format(lb) for lb in reg_lbs]
    agent_ids += ["hgcbc_l2_mlp_lb{}".format(lb) for lb in reg_lbs]

    hsp_eps = [0.1]
    hsp_lbs = [0.1]
    agent_ids += ["hgcbc_hispofix_mlp_heps{}_leps{}_lb{}_l2".format(heps, leps, lb) for heps in hsp_eps for leps in hsp_eps for lb in hsp_lbs]

    agent_id_to_algo = {
        "hgcbc_scratch_1_mlp": "HBC",
        "hgcbc_scratch_n_mlp": "HBC",
        "hgcbc_finetune_1_mlp": "HBC",
        "hgcbc_finetune_n_mlp": "HBC",
        "hgcbc_freeze_mlp": "HBC",
        "hgcbc_pnn_mlp": "HBC",
        "hgcbc_replay_mlp": "HBC",
    }

    for lb in reg_lbs:
        agent_id_to_algo["hgcbc_ewc_mlp_lb{}".format(lb)] = "HBC"
        agent_id_to_algo["hgcbc_l2_mlp_lb{}".format(lb)] = "HBC"
    
    for lb in hsp_lbs:
        for heps in hsp_eps:
            for leps in hsp_eps:
                agent_id_to_algo["hgcbc_hispofix_mlp_heps{}_leps{}_lb{}_2".format(heps, leps, lb)] = "HBC"

    agent_id_to_method = {
        "hgcbc_scratch_1_mlp": "SC1",
        "hgcbc_scratch_n_mlp": "SCN",
        "hgcbc_finetune_1_mlp": "FT1",
        "hgcbc_finetune_n_mlp": "FTN",
        "hgcbc_freeze_mlp": "FRZ",
        "hgcbc_pnn_mlp": "PNN",
        "hgcbc_replay_mlp": "RPL",
    }

    for lb in reg_lbs:
        agent_id_to_method["hgcbc_ewc_mlp_lb{}".format(lb)] = "EWC (λ={})".format(lb)
        agent_id_to_method["hgcbc_l2_mlp_lb{}".format(lb)] = "L2 (λ={})".format(lb)

    for lb in hsp_lbs:
        for heps in hsp_eps:
            for leps in hsp_eps:
                agent_id_to_method["hgcbc_hispofix_mlp_heps{}_leps{}_lb{}_l2".format(heps, leps, lb)] = "HiSPO (hε={}, lε={}, λ={})".format(heps, leps, lb)

#### 1.C. Data

In [3]:
def generate_data():

    # ROUGH DATA
    ############

    data = []

    for stream_name in stream_names:

        for agent_id in agent_ids:
            
            for seed in seeds:

                experiment_folder_path = os.path.join(all_experiments_folder_path, stream_name, agent_id, f"seed_{seed}")
                path_to_logs = os.path.join(experiment_folder_path, "logs")

                if not os.path.exists(path_to_logs): continue

                events_files = os.listdir(path_to_logs)
                events_files = [f for f in events_files if "events.out.tfevents" in f]

                all_metrics = []

                for events_file in events_files:

                    metrics = extract_metrics_from_event_file(path_to_logs + "/" + events_file)

                    for tag,step,value in metrics:

                        if tag.startswith("EVALUATIONS"):
                            
                            if step == max_gradient_steps:

                                train_task_idx = int(tag.split("/")[0].split("_")[1][1:])
                                eval_task_idx = int(tag.split("/")[1].split("_")[0][1:])
                                metric_name = "_".join(tag.split("/")[1].split("_")[1:])

                                if (train_task_idx <= n_tasks) and (eval_task_idx <= n_tasks):

                                    if "success" in metric_name:

                                        all_metrics.append({
                                            "stream_name": stream_name,
                                            "agent_id": agent_id,
                                            "seed": seed,
                                            "train_task_idx": train_task_idx,
                                            "eval_task_idx": eval_task_idx,
                                            "metric": metric_name,
                                            "value": value,
                                        })
                        
                        elif tag.startswith("SIZE"):

                            if step <= n_tasks:

                                metric = tag.split("/")[1]

                                for i in range(n_tasks):

                                    all_metrics.append({
                                        "stream_name": stream_name,
                                        "agent_id": agent_id,
                                        "seed": seed,
                                        "train_task_idx": step,
                                        "eval_task_idx": i+1,
                                        "metric": metric,
                                        "value": value,
                                    })
                        
                        elif tag.startswith("HIGH_SIZE"):

                            if step <= n_tasks:

                                metric = "high_" + tag.split("/")[1]

                                for i in range(n_tasks):

                                    all_metrics.append({
                                        "stream_name": stream_name,
                                        "agent_id": agent_id,
                                        "seed": seed,
                                        "train_task_idx": step,
                                        "eval_task_idx": i+1,
                                        "metric": metric,
                                        "value": value,
                                    })
                        
                        elif tag.startswith("LOW_SIZE"):

                            if step <= n_tasks:

                                metric = "low_" + tag.split("/")[1]

                                for i in range(n_tasks):

                                    all_metrics.append({
                                        "stream_name": stream_name,
                                        "agent_id": agent_id,
                                        "seed": seed,
                                        "train_task_idx": step,
                                        "eval_task_idx": i+1,
                                        "metric": metric,
                                        "value": value,
                                    })
                    
                    try:
                    
                        df = pd.DataFrame(all_metrics)
                        df = df.pivot_table(index=["stream_name", "agent_id", "seed", "train_task_idx", "eval_task_idx"], columns="metric", values="value")
                        df = df.reset_index(drop=False)
                    
                    except:

                        print(f"Error with {experiment_folder_path}")
                    
                    data.append(df)

    data = pd.concat(data)
    data = data.reset_index(drop=True)
    data = data.rename(columns={"mean_success": "success"})

    # NORMALIZED SIZE
    #################

    high_ref_size = data[
        (data["agent_id"] == "hgcbc_scratch_1_mlp") & 
        (data["stream_name"] == stream_names[0]) &
        (data["seed"] == 100) &
        (data["train_task_idx"] == 1) &
        (data["eval_task_idx"] == 1)
    ]["high_inference_size"]

    data["high_inference_size"] = data["high_inference_size"] / high_ref_size.values
    data["high_total_size"] = data["high_total_size"] / high_ref_size.values

    low_ref_size = data[
        (data["agent_id"] == "hgcbc_scratch_1_mlp") & 
        (data["stream_name"] == stream_names[0]) &
        (data["seed"] == 100) &
        (data["train_task_idx"] == 1) &
        (data["eval_task_idx"] == 1)
    ]["low_inference_size"]

    data["low_inference_size"] = data["low_inference_size"] / low_ref_size.values
    data["low_total_size"] = data["low_total_size"] / low_ref_size.values

    total_ref_size = data[
        (data["agent_id"] == "hgcbc_scratch_1_mlp") & 
        (data["stream_name"] == stream_names[0]) &
        (data["seed"] == 100) &
        (data["train_task_idx"] == 1) &
        (data["eval_task_idx"] == 1)
    ]["inference_size"]

    data["inference_size"] = data["inference_size"] / total_ref_size.values
    data["total_size"] = data["total_size"] / total_ref_size.values

    # FILTERING
    ###########

    # group by agent_id, stream_name, and seed
    grouped = data.groupby(["agent_id", "stream_name", "seed"])
    
    # identify valid agents with full task coverage (train_task_idx)
    valid_agents = []
    for (agent, stream, seed), group in grouped:
        train_task_indices = set(group["train_task_idx"])
        if set(range(1, n_tasks + 1)).issubset(train_task_indices):
            valid_agents.append((agent, stream, seed))
    
    # filter the dataframe
    filtered_data = data.set_index(["agent_id", "stream_name", "seed"])
    filtered_data = filtered_data.loc[valid_agents].reset_index()

    data = filtered_data
    
    data = data[[
        "agent_id", "stream_name", "seed", 
        "train_task_idx", "eval_task_idx", 
        "success", 
        "inference_size", "total_size",
        "high_inference_size", "high_total_size",
        "low_inference_size", "low_total_size",
    ]]

    # remove duplicate rows
    data = data.drop_duplicates()

    return data

In [4]:
data = generate_data()

#### 1.D. Metrics

In [5]:
def generate_metrics(data: pd.DataFrame) -> pd.DataFrame:

    group_cols = ["stream_name", "agent_id", "seed"]

    processed_data = []

    for (stream, agent, seed), df_sub in data.groupby(group_cols):

        for N in range(1, n_tasks+1):

            # pivot to get perf_matrix
            perf_matrix = df_sub.pivot(index="train_task_idx", columns="eval_task_idx", values="success")
                        
            # Performance (mean for 1 to N)
            PER = 0.0
            for k in range(1, N+1):
                PER += perf_matrix.loc[N, k]
            PER /= N
            
            # BWT
            bwt_sum = 0.0
            for k in range(1, N+1):
                bwt_sum += perf_matrix.loc[N, k] - perf_matrix.loc[k, k]
            BWT = bwt_sum / N
            
            # For FWT, we also need the scratch model for the same (stream, seed)
            # But agent_id='gcbc_scratch_1_mlp'. We'll do a separate query:
            df_scratch_sub = data.query(f"stream_name == '{stream}' & seed == {seed} & agent_id == 'hgcbc_scratch_1_mlp'")
            scratch_matrix = df_scratch_sub.pivot(index="train_task_idx", columns="eval_task_idx", values="success")
            
            fwt_sum = 0.0
            for k in range(1, N+1):
                fwt_sum += perf_matrix.loc[k, k] - scratch_matrix.loc[k, k]
            FWT = fwt_sum / N
            
            # MEM
            final_model_row = df_sub.query("train_task_idx == @N").iloc[0]
            
            processed_data.append({
                "stream_name": stream,
                "agent_id": agent,
                "seed": seed,
                "train_task_idx": N,
                "PER": PER,
                "BWT": BWT,
                "FWT": FWT,
                "HIGH_MEM_inference": final_model_row["high_inference_size"],
                "HIGH_MEM_total": final_model_row["high_total_size"],
                "LOW_MEM_inference": final_model_row["low_inference_size"],
                "LOW_MEM_total": final_model_row["low_total_size"],
                "MEM_inference": final_model_row["inference_size"],
                "MEM_total": final_model_row["total_size"],
            })

    processed_data = pd.DataFrame(processed_data)

    processed_data["method"] = processed_data["agent_id"].apply(lambda x: agent_id_to_method[x])

    processed_data = processed_data[[
        "stream_name", "method", "seed",
        "train_task_idx",
        "PER", "BWT", "FWT", 
        "HIGH_MEM_inference", "HIGH_MEM_total",
        "LOW_MEM_inference", "LOW_MEM_total",
        "MEM_inference", "MEM_total"
    ]]

    return processed_data

In [6]:
metrics = generate_metrics(data)

___

### **2. Tables by Streams**

#### 2.A. Stream 1

In [7]:
metrics_1 = metrics[metrics["stream_name"] == stream_names[0]]
metrics_1 = metrics_1[metrics_1["train_task_idx"] == n_tasks]

# average over seeds
metrics_1 = metrics_1.groupby(["stream_name", "method"]).agg({
    "PER": ["mean", "std"],
    "BWT": ["mean", "std"],
    "FWT": ["mean", "std"],
    "HIGH_MEM_inference": ["mean", "std"],
    "HIGH_MEM_total": ["mean", "std"],
    "LOW_MEM_inference": ["mean", "std"],
    "LOW_MEM_total": ["mean", "std"],
    "MEM_inference": ["mean", "std"],
    "MEM_total": ["mean", "std"],
}).reset_index()
metrics_1.columns = ["_".join(col).strip() for col in metrics_1.columns.values]
metrics_1 = metrics_1.sort_values("PER_mean", ascending=False)

# find 1st iteration of EWC (the best EWC for any lambda) and remove the other EWC
ewc_idx = metrics_1[metrics_1["method_"].str.contains("EWC")].index[0]
ewc_idx = metrics_1[(metrics_1["method_"].str.contains("EWC")) & (metrics_1.index != ewc_idx)].index
metrics_1 = metrics_1.drop(ewc_idx)
# find 1st iteration of L2 (the best L2 for any lambda) and remove the other L2
l2_idx = metrics_1[metrics_1["method_"].str.contains("L2")].index[0]
l2_idx = metrics_1[(metrics_1["method_"].str.contains("L2")) & (metrics_1.index != l2_idx)].index
metrics_1 = metrics_1.drop(l2_idx)
# find 1st iteration of HiSPO (the best HiSPO for any epsilon and lambda) and remove the other HiSPO
hispo_idx = metrics_1[metrics_1["method_"].str.contains("HiSPO")].index[0]
hispo_idx = metrics_1[(metrics_1["method_"].str.contains("HiSPO")) & (metrics_1.index != hispo_idx)].index
metrics_1 = metrics_1.drop(hispo_idx)

# show
metrics_1["PER_mean"] = (metrics_1["PER_mean"] * 100).round(1)
metrics_1["PER_std"] = (metrics_1["PER_std"] * 100).round(1)
metrics_1["BWT_mean"] = (metrics_1["BWT_mean"] * 100).round(1)
metrics_1["BWT_std"] = (metrics_1["BWT_std"] * 100).round(1)
metrics_1["FWT_mean"] = (metrics_1["FWT_mean"] * 100).round(1)
metrics_1["FWT_std"] = (metrics_1["FWT_std"] * 100).round(1)
metrics_1["HIGH_MEM_inference_mean"] = metrics_1["HIGH_MEM_inference_mean"].round(1)
metrics_1["HIGH_MEM_inference_std"] = metrics_1["HIGH_MEM_inference_std"].round(1)
metrics_1["HIGH_MEM_total_mean"] = metrics_1["HIGH_MEM_total_mean"].round(1)
metrics_1["HIGH_MEM_total_std"] = metrics_1["HIGH_MEM_total_std"].round(1)
metrics_1["LOW_MEM_inference_mean"] = metrics_1["LOW_MEM_inference_mean"].round(1)
metrics_1["LOW_MEM_inference_std"] = metrics_1["LOW_MEM_inference_std"].round(1)
metrics_1["LOW_MEM_total_mean"] = metrics_1["LOW_MEM_total_mean"].round(1)
metrics_1["LOW_MEM_total_std"] = metrics_1["LOW_MEM_total_std"].round(1)
metrics_1["MEM_inference_mean"] = metrics_1["MEM_inference_mean"].round(1)
metrics_1["MEM_inference_std"] = metrics_1["MEM_inference_std"].round(1)
metrics_1["MEM_total_mean"] = metrics_1["MEM_total_mean"].round(1)

metrics_1

,stream_name_,method_,PER_mean,PER_std,BWT_mean,BWT_std,FWT_mean,FWT_std,HIGH_MEM_inference_mean,HIGH_MEM_inference_std,HIGH_MEM_total_mean,HIGH_MEM_total_std,LOW_MEM_inference_mean,LOW_MEM_inference_std,LOW_MEM_total_mean,LOW_MEM_total_std,MEM_inference_mean,MEM_inference_std,MEM_total_mean,MEM_total_std
14,simpletown_topological_1,PNN,98.8,0.1,0.0,0.0,1.9,2.0,9.9,0.0,9.9,0.0,9.9,0.0,9.9,0.0,9.9,0.0,9.9,0.0
15,simpletown_topological_1,RPL,97.8,0.6,0.9,0.1,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
7,simpletown_topological_1,FTN,97.5,1.2,0.0,0.0,0.6,1.7,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0
8,simpletown_topological_1,"HiSPO (hε=0.1, lε=0.1, λ=0.1)",97.0,0.0,0.0,0.0,0.1,2.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0
17,simpletown_topological_1,SCN,96.9,2.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0
2,simpletown_topological_1,EWC (λ=1.0),94.8,1.6,-2.4,0.8,0.3,1.3,1.0,0.0,9.0,0.0,1.0,0.0,9.0,0.0,1.0,0.0,9.0,0.0
6,simpletown_topological_1,FT1,94.1,0.8,-3.4,0.8,0.6,1.7,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
11,simpletown_topological_1,L2 (λ=1.0),93.9,2.3,-1.7,3.5,-1.3,0.6,1.0,0.0,5.0,0.0,1.0,0.0,5.0,0.0,1.0,0.0,5.0,0.0
16,simpletown_topological_1,SC1,90.6,3.8,-6.3,5.1,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
5,simpletown_topological_1,FRZ,84.6,1.3,0.0,0.0,-12.3,2.7,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


#### 2.B. Stream 2

In [8]:
metrics_2 = metrics[metrics["stream_name"] == stream_names[1]]
metrics_2 = metrics_2[metrics_2["train_task_idx"] == n_tasks]

# average over seeds
metrics_2 = metrics_2.groupby(["stream_name", "method"]).agg({
    "PER": ["mean", "std"],
    "BWT": ["mean", "std"],
    "FWT": ["mean", "std"],
    "HIGH_MEM_inference": ["mean", "std"],
    "HIGH_MEM_total": ["mean", "std"],
    "LOW_MEM_inference": ["mean", "std"],
    "LOW_MEM_total": ["mean", "std"],
    "MEM_inference": ["mean", "std"],
    "MEM_total": ["mean", "std"],
}).reset_index()
metrics_2.columns = ["_".join(col).strip() for col in metrics_2.columns.values]
metrics_2 = metrics_2.sort_values("PER_mean", ascending=False)

# find 1st iteration of EWC (the best EWC for any lambda) and remove the other EWC
ewc_idx = metrics_2[metrics_2["method_"].str.contains("EWC")].index[0]
ewc_idx = metrics_2[(metrics_2["method_"].str.contains("EWC")) & (metrics_2.index != ewc_idx)].index
metrics_2 = metrics_2.drop(ewc_idx)
# find 1st iteration of L2 (the best L2 for any lambda) and remove the other L2
l2_idx = metrics_2[metrics_2["method_"].str.contains("L2")].index[0]
l2_idx = metrics_2[(metrics_2["method_"].str.contains("L2")) & (metrics_2.index != l2_idx)].index
metrics_2 = metrics_2.drop(l2_idx)
# find 1st iteration of HiSPO (the best HiSPO for any epsilon and lambda) and remove the other HiSPO
hispo_idx = metrics_2[metrics_2["method_"].str.contains("HiSPO")].index[0]
hispo_idx = metrics_2[(metrics_2["method_"].str.contains("HiSPO")) & (metrics_2.index != hispo_idx)].index
metrics_2 = metrics_2.drop(hispo_idx)

# show
metrics_2["PER_mean"] = (metrics_2["PER_mean"] * 100).round(1)
metrics_2["PER_std"] = (metrics_2["PER_std"] * 100).round(1)
metrics_2["BWT_mean"] = (metrics_2["BWT_mean"] * 100).round(1)
metrics_2["BWT_std"] = (metrics_2["BWT_std"] * 100).round(1)
metrics_2["FWT_mean"] = (metrics_2["FWT_mean"] * 100).round(1)
metrics_2["FWT_std"] = (metrics_2["FWT_std"] * 100).round(1)
metrics_2["HIGH_MEM_inference_mean"] = metrics_2["HIGH_MEM_inference_mean"].round(1)
metrics_2["HIGH_MEM_inference_std"] = metrics_2["HIGH_MEM_inference_std"].round(1)
metrics_2["HIGH_MEM_total_mean"] = metrics_2["HIGH_MEM_total_mean"].round(1)
metrics_2["HIGH_MEM_total_std"] = metrics_2["HIGH_MEM_total_std"].round(1)
metrics_2["LOW_MEM_inference_mean"] = metrics_2["LOW_MEM_inference_mean"].round(1)
metrics_2["LOW_MEM_inference_std"] = metrics_2["LOW_MEM_inference_std"].round(1)
metrics_2["LOW_MEM_total_mean"] = metrics_2["LOW_MEM_total_mean"].round(1)
metrics_2["LOW_MEM_total_std"] = metrics_2["LOW_MEM_total_std"].round(1)
metrics_2["MEM_inference_mean"] = metrics_2["MEM_inference_mean"].round(1)
metrics_2["MEM_inference_std"] = metrics_2["MEM_inference_std"].round(1)
metrics_2["MEM_total_mean"] = metrics_2["MEM_total_mean"].round(1)

metrics_2

,stream_name_,method_,PER_mean,PER_std,BWT_mean,BWT_std,FWT_mean,FWT_std,HIGH_MEM_inference_mean,HIGH_MEM_inference_std,HIGH_MEM_total_mean,HIGH_MEM_total_std,LOW_MEM_inference_mean,LOW_MEM_inference_std,LOW_MEM_total_mean,LOW_MEM_total_std,MEM_inference_mean,MEM_inference_std,MEM_total_mean,MEM_total_std
14,simpletown_topological_2,PNN,98.1,0.5,0.0,0.0,1.9,1.0,9.9,0.0,9.9,0.0,9.9,0.0,9.9,0.0,9.9,0.0,9.9,0.0
8,simpletown_topological_2,"HiSPO (hε=0.1, lε=0.1, λ=0.1)",97.1,2.2,0.0,0.0,0.9,1.8,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0
7,simpletown_topological_2,FTN,96.8,0.3,0.0,0.0,0.7,0.8,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0
17,simpletown_topological_2,SCN,96.2,0.6,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0
15,simpletown_topological_2,RPL,91.8,1.7,-1.6,0.6,-2.7,1.8,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,simpletown_topological_2,EWC (λ=0.1),83.2,4.8,-13.5,4.8,0.5,0.4,1.0,0.0,9.0,0.0,1.0,0.0,9.0,0.0,1.0,0.0,9.0,0.0
6,simpletown_topological_2,FT1,81.2,3.3,-15.7,3.4,0.7,0.8,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
11,simpletown_topological_2,L2 (λ=1.0),81.1,4.1,-8.5,2.4,-6.6,2.3,1.0,0.0,5.0,0.0,1.0,0.0,5.0,0.0,1.0,0.0,5.0,0.0
16,simpletown_topological_2,SC1,74.8,0.6,-21.3,0.8,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
5,simpletown_topological_2,FRZ,64.1,1.5,0.0,0.0,-32.1,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


#### 2.C. Both

In [9]:
metrics_all = metrics
metrics_all = metrics_all[metrics_all["train_task_idx"] == n_tasks]

# average over seeds
metrics_all = metrics_all.groupby(["method"]).agg({
    "PER": ["mean", "std"],
    "BWT": ["mean", "std"],
    "FWT": ["mean", "std"],
    "HIGH_MEM_inference": ["mean", "std"],
    "HIGH_MEM_total": ["mean", "std"],
    "LOW_MEM_inference": ["mean", "std"],
    "LOW_MEM_total": ["mean", "std"],
    "MEM_inference": ["mean", "std"],
    "MEM_total": ["mean", "std"],
}).reset_index()
metrics_all.columns = ["_".join(col).strip() for col in metrics_all.columns.values]
metrics_all = metrics_all.sort_values("PER_mean", ascending=False)

# find 1st iteration of EWC (the best EWC for any lambda) and remove the other EWC
ewc_idx = metrics_all[metrics_all["method_"].str.contains("EWC")].index[0]
ewc_idx = metrics_all[(metrics_all["method_"].str.contains("EWC")) & (metrics_all.index != ewc_idx)].index
metrics_all = metrics_all.drop(ewc_idx)
# find 1st iteration of L2 (the best L2 for any lambda) and remove the other L2
l2_idx = metrics_all[metrics_all["method_"].str.contains("L2")].index[0]
l2_idx = metrics_all[(metrics_all["method_"].str.contains("L2")) & (metrics_all.index != l2_idx)].index
metrics_all = metrics_all.drop(l2_idx)
# find 1st iteration of HiSPO (the best HiSPO for any epsilon and lambda) and remove the other HiSPO
hispo_idx = metrics_all[metrics_all["method_"].str.contains("HiSPO")].index[0]
hispo_idx = metrics_all[(metrics_all["method_"].str.contains("HiSPO")) & (metrics_all.index != hispo_idx)].index
metrics_all = metrics_all.drop(hispo_idx)

# show
metrics_all["PER_mean"] = (metrics_all["PER_mean"] * 100).round(1)
metrics_all["PER_std"] = (metrics_all["PER_std"] * 100).round(1)
metrics_all["BWT_mean"] = (metrics_all["BWT_mean"] * 100).round(1)
metrics_all["BWT_std"] = (metrics_all["BWT_std"] * 100).round(1)
metrics_all["FWT_mean"] = (metrics_all["FWT_mean"] * 100).round(1)
metrics_all["FWT_std"] = (metrics_all["FWT_std"] * 100).round(1)
metrics_all["HIGH_MEM_inference_mean"] = metrics_all["HIGH_MEM_inference_mean"].round(1)
metrics_all["HIGH_MEM_inference_std"] = metrics_all["HIGH_MEM_inference_std"].round(1)
metrics_all["HIGH_MEM_total_mean"] = metrics_all["HIGH_MEM_total_mean"].round(1)
metrics_all["HIGH_MEM_total_std"] = metrics_all["HIGH_MEM_total_std"].round(1)
metrics_all["LOW_MEM_inference_mean"] = metrics_all["LOW_MEM_inference_mean"].round(1)
metrics_all["LOW_MEM_inference_std"] = metrics_all["LOW_MEM_inference_std"].round(1)
metrics_all["LOW_MEM_total_mean"] = metrics_all["LOW_MEM_total_mean"].round(1)
metrics_all["LOW_MEM_total_std"] = metrics_all["LOW_MEM_total_std"].round(1)
metrics_all["MEM_inference_mean"] = metrics_all["MEM_inference_mean"].round(1)
metrics_all["MEM_inference_std"] = metrics_all["MEM_inference_std"].round(1)
metrics_all["MEM_total_mean"] = metrics_all["MEM_total_mean"].round(1)

metrics_all

,method_,PER_mean,PER_std,BWT_mean,BWT_std,FWT_mean,FWT_std,HIGH_MEM_inference_mean,HIGH_MEM_inference_std,HIGH_MEM_total_mean,HIGH_MEM_total_std,LOW_MEM_inference_mean,LOW_MEM_inference_std,LOW_MEM_total_mean,LOW_MEM_total_std,MEM_inference_mean,MEM_inference_std,MEM_total_mean,MEM_total_std
14,PNN,98.5,0.5,0.0,0.0,1.9,1.4,9.9,0.0,9.9,0.0,9.9,0.0,9.9,0.0,9.9,0.0,9.9,0.000000
7,FTN,97.2,0.9,0.0,0.0,0.6,1.2,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.000000
8,"HiSPO (hε=0.1, lε=0.1, λ=0.1)",97.0,1.4,0.0,0.0,0.5,1.8,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.000003
17,SCN,96.5,1.4,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.000000
15,RPL,94.8,3.5,-0.3,1.4,-1.4,2.3,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.000000
3,EWC (λ=10.0),88.4,7.5,-8.5,7.0,0.3,1.1,1.0,0.0,9.0,0.0,1.0,0.0,9.0,0.0,1.0,0.0,9.0,0.000000
6,FT1,87.6,7.4,-9.5,7.1,0.6,1.2,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.000000
11,L2 (λ=1.0),87.5,7.6,-5.1,4.6,-4.0,3.3,1.0,0.0,5.0,0.0,1.0,0.0,5.0,0.0,1.0,0.0,5.0,0.000000
16,SC1,82.7,9.0,-13.8,8.8,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.000000
5,FRZ,74.3,11.3,0.0,0.0,-22.2,11.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.000000


In [11]:
metrics_all_simplified = metrics_all.copy(deep=True)
# simplify method names (split by space and take the first word)
metrics_all_simplified["method_"] = metrics_all_simplified["method_"].apply(lambda x: x.split(" ")[0])
# rename
metrics_all_simplified = metrics_all_simplified.rename(columns={"method_": "method"})
# save
metrics_all_simplified.to_csv("./analysis.csv", index=False)
# show
metrics_all_simplified

,method,PER_mean,PER_std,BWT_mean,BWT_std,FWT_mean,FWT_std,HIGH_MEM_inference_mean,HIGH_MEM_inference_std,HIGH_MEM_total_mean,HIGH_MEM_total_std,LOW_MEM_inference_mean,LOW_MEM_inference_std,LOW_MEM_total_mean,LOW_MEM_total_std,MEM_inference_mean,MEM_inference_std,MEM_total_mean,MEM_total_std
14,PNN,98.5,0.5,0.0,0.0,1.9,1.4,9.9,0.0,9.9,0.0,9.9,0.0,9.9,0.0,9.9,0.0,9.9,0.000000
7,FTN,97.2,0.9,0.0,0.0,0.6,1.2,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.000000
8,HiSPO,97.0,1.4,0.0,0.0,0.5,1.8,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.000003
17,SCN,96.5,1.4,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.000000
15,RPL,94.8,3.5,-0.3,1.4,-1.4,2.3,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.000000
3,EWC,88.4,7.5,-8.5,7.0,0.3,1.1,1.0,0.0,9.0,0.0,1.0,0.0,9.0,0.0,1.0,0.0,9.0,0.000000
6,FT1,87.6,7.4,-9.5,7.1,0.6,1.2,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.000000
11,L2,87.5,7.6,-5.1,4.6,-4.0,3.3,1.0,0.0,5.0,0.0,1.0,0.0,5.0,0.0,1.0,0.0,5.0,0.000000
16,SC1,82.7,9.0,-13.8,8.8,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.000000
5,FRZ,74.3,11.3,0.0,0.0,-22.2,11.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.000000


___
___